In [3]:
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm.auto import tqdm
tqdm.pandas()
import logging

In [3]:
# URL of the CSV file
bussgeld_url = "https://offenedaten-koeln.de/sites/default/files/Bussgeld_2021.csv"
# Read the CSV file using pandas
bussgeld = pd.read_csv(bussgeld_url,  delimiter=';', on_bad_lines='skip')

C:\Users\anagh\AppData\Local\Temp\ipykernel_24820\1956291180.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  bussgeld = pd.read_csv(bussgeld_url,  delimiter=';', on_bad_lines='skip')


In [4]:
bussgeld

,Jahr,Monat,datum_von,kennzeichen1,fahrzeugart,fabrikat,strasse,hausnummer,tatbestand1,tatbestand2,tatbestand3
0,2021,1,2021-01-14 08:32:00.000,MG,PKW,FIAT,BEETHOVENSTR.,Mai 13,141324,NaN,NaN
1,2021,1,2021-01-14 13:23:00.000,BM,PKW,OPEL,HOHENSTAUFENRING,61,141322,NaN,NaN
2,2021,1,2021-01-14 13:23:00.000,K,PKW,VW-VOLKSWAGEN,HOHENSTAUFENRING,57A,141322,NaN,NaN
3,2021,1,2021-01-14 09:25:00.000,K,PKW,FORD,LINDENSTR.,52,141324,NaN,NaN
4,2021,1,2021-01-14 14:25:00.000,M,PKW,VW-VOLKSWAGEN,MOLTKESTR.,NaN,141322,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
589084,2021,12,2021-12-17 10:06:00.000,BM,PKW,SMART,DÜRENER STR.,240,113140,NaN,NaN
589085,2021,12,2021-12-17 10:06:00.000,BM,PKW,VW-VOLKSWAGEN,DÜRENER STR.,238,113143,NaN,NaN
589086,2021,12,2021-12-17 10:00:00.000,PB,PKW,BMW,DÜRENER STR.,244,113144,NaN,NaN
589087,2021,12,2021-12-14 11:19:00.000,K,PKW,MERCEDES,ZÜLPICHER STR.,311,142284,NaN,NaN


In [100]:
# URL of the CSV file
lsa_url1 = "https://offenedaten-koeln.de/sites/default/files/2023_LSA_Koeln.csv"            
lsa_url2 = "https://offenedaten-koeln.de/sites/default/files/20181116_LSA_Ko%CC%88ln.csv"
lsa_url3 = "https://offenedaten-koeln.de/sites/default/files/20131118_lsa_koordinaten_wgs84.csv"
# Read the CSV file using pandas
lsa1 = pd.read_csv(lsa_url1,  delimiter=';', on_bad_lines='skip')
lsa2 = pd.read_csv(lsa_url2,  delimiter=';', on_bad_lines='skip', encoding= 'latin-1')
lsa3 = pd.read_csv(lsa_url3,  delimiter=';', on_bad_lines='skip')

In [101]:
lsa1.rename(columns={'X-Koordinate': 'X', 'Y-Koordinate': 'Y'}, inplace=True)

In [102]:
# Identify the unique column present in all three datasets
unique_column = 'LSA-Nr.'

# Select the four columns to keep from each dataset
columns_to_keep = ['LSA-Nr.', 'Standort', 'X', 'Y']

# Keep only the selected columns in each dataset
dataset1 = lsa1[columns_to_keep]
dataset2 = lsa2[columns_to_keep]
dataset3 = lsa3[columns_to_keep]


In [103]:
dataset2['LSA-Nr.'] = dataset2['LSA-Nr.'].astype(str)
dataset3['LSA-Nr.'] = dataset3['LSA-Nr.'].astype(str)
merged_dataset = pd.concat([dataset1, dataset2, dataset3], ignore_index=True)
merged_dataset.shape

C:\Users\anagh\AppData\Local\Temp\ipykernel_33936\1743010352.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset2['LSA-Nr.'] = dataset2['LSA-Nr.'].astype(str)


(2903, 4)

In [104]:
merged_dataset.drop_duplicates(subset=[unique_column], inplace=True)

In [105]:
merged_dataset.shape

(1969, 4)

In [106]:
merged_dataset.to_csv('LSA.csv', index=False)  

In [107]:
# Create a geocoder instance
geolocator = Nominatim(user_agent="saki")

In [34]:
def getLatAndLong(row):
    latitude = 0.00
    longitude = 0.00
    strasse = row['strasse']
    hausnummer = row['hausnummer']
    street_name = str(strasse) + " " + str(hausnummer)
    try:
        location = geolocator.geocode(f"{street_name}, Köln, Germany")
        latitude = location.latitude
        longitude = location.longitude
        return latitude, longitude
    except:
        logging.info("Trying without hausnummer")
        street_name = str(strasse)
        try:
            location = geolocator.geocode(f"{street_name}, Köln, Germany")
            latitude = location.latitude
            longitude = location.longitude
            return latitude, longitude
        except:
            logging.info("No geocode found")
            return latitude, longitude

In [35]:
bussgeld = bussgeld[bussgeld["Monat "] <= 3]

In [37]:
bussgeld['latAndlong'] = bussgeld.progress_apply(getLatAndLong, axis=1)

  0%|          | 0/147018 [00:00<?, ?it/s]

C:\Users\anagh\AppData\Local\Temp\ipykernel_3304\3248858535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bussgeld['latAndlong'] = bussgeld.progress_apply(getLatAndLong, axis=1)


In [38]:
bussgeld.to_csv('bussgeldWithLat.csv')  